In [ ]:
import requests
import urllib.parse
from langchain.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import JsonOutputParser
from pydantic import BaseModel
from langchain.chains import LLMChain

# Define a model for keyword extraction
class SearchKeyword(BaseModel):
    keyword: str

# Initialize LLM with centralized server configuration
from dotenv import load_dotenv
load_dotenv()
import os

llm = ChatOpenAI(
    model='gpt-4o-mini',
    base_url= os.getenv("url"),  
    api_key= os.getenv("api_key"),  
    max_tokens=1000  
)

def clean_keyword(keyword: str):
    return keyword.strip().lower()

def process_user_query(user_query):
    try:
        parser = JsonOutputParser(pydantic_object=SearchKeyword)
        format_instructions = """
        คุณต้องกรองคำสำคัญจากคำขอของผู้ใช้.
        คำขอของผู้ใช้คือ: {user_query}
        คำสำคัญที่คุณต้องการหาคือสิ่งที่เกี่ยวข้องกับการค้นหาหรือการกระทำที่ผู้ใช้ต้องการ เช่น:
        - ต้องการทานข้าวหรือแวะพักทานอาหาร: "ร้านอาหาร"
        - ต้องการทานกาแฟ: "ร้านกาแฟ"
        - หากผู้ใช้ต้องการหาห้องน้ำ: "ห้องน้ำ"
        - หากผู้ใช้ต้องการเติมน้ำมัน: "ปั๊มน้ำมัน"
        - หากผู้ใช้ต้องการซื้อของฝาก: "ร้านขายของฝาก"
        - หากผู้ใช้ต้องการซื้อของ: "ร้านสะดวกซื้อ"
        - หากผู้ใช้ต้องการสถานที่แวะพักระหว่างการเดินทาง: "สถานที่พัก"
        กรุณาตอบคำสำคัญที่พบในคำขอนี้ในรูปแบบ JSON ตามตัวอย่างนี้:
        {
            "keyword": "<extracted_keyword>"
        }
        """
        
        prompt = PromptTemplate(
            template="""\
            ## คุณมีหน้าที่กรองคำสำคัญจากคำขอผู้ใช้.

            # คำขอผู้ใช้: {user_query}

            # Your response should be structured as follows:
            {format_instructions}
            """,
            input_variables=["user_query"],
            partial_variables={"format_instructions": format_instructions},
        )

        chain = prompt | llm | parser
        event = chain.invoke({"user_query": user_query})

        if event:
            keyword = event.get('keyword', '')
            cleaned_keyword = clean_keyword(keyword)
            if cleaned_keyword:
                return cleaned_keyword
    except Exception as e:
        print(f"Error processing user query: {e}")
        return None
    
def convert_locations(user_location, user_destination):
    flat, flon = user_location
    tlat, tlon = user_destination
    return flon, flat, tlon, tlat

In [ ]:
def search_logdo_map_api(keyword, user_location, radius):
    try:
        base_url = "https://api.longdo.com/mapsearch/json/search?"
        params = {
            'key': os.getenv("key_longdo"),
            'lon': user_location[1],
            'lat': user_location[0],
            'radius': radius * 1000,
            'keyword': keyword
        }
        full_url = base_url + urllib.parse.urlencode(params)

        response = requests.get(full_url)
        response.raise_for_status()
        return response.json()
    except requests.exceptions.RequestException as e:
        print(f"Error fetching data from Longdo API: {e}")
    return None

In [ ]:
def get_route_data(flon, flat, tlon, tlat):
    """
    Get the route data from the Longdo API based on the given start and end points.

    Args:
        flon (float): Longitude of the start point.
        flat (float): Latitude of the start point.
        tlon (float): Longitude of the destination point.
        tlat (float): Latitude of the destination point.

    Returns:
        dict: Route data returned by the Longdo API.
    """
    try:
        base_url = "https://api.longdo.com/RouteService/json/route/guide?"
        params = {
            'key': os.getenv("key_longdo"),
            'flon': flon,
            'flat': flat,
            'tlon': tlon,
            'tlat': tlat
        }
        full_url = base_url + urllib.parse.urlencode(params)

        response = requests.get(full_url)
        response.raise_for_status()
        route_data = response.json()
    
        return route_data
    except requests.exceptions.RequestException as e:
        print(f"Error fetching route data: {e}")
    return None


In [4]:
def search_places_of_interest(route_data, keyword, radius):
    """
    Search for places of interest along the route using the Longdo Map API and geocode the addresses.

    Args:
        route_data (dict): Route data containing the list of guide points.
        keyword (str): Keyword for the type of places to search for.
        radius (int): The radius to search for places (in kilometers).

    Returns:
        list: List of places of interest along the route, each with latitude and longitude.
    """
    places_of_interest = []

    for point in route_data.get('data', [])[0].get('guide', []):
        name = point.get('name')
        if name:
            try:
                address = name
                # Use Nominatim to geocode the address
                url = f'https://nominatim.openstreetmap.org/search?q={urllib.parse.quote(address)}&format=json'
                headers = {'User-Agent': 'MyGeocodingApp/1.0'}

                response = requests.get(url, headers=headers)
                response.raise_for_status()

                data = response.json()
                if data:
                    # Extract latitude and longitude of the location
                    latitude = float(data[0]["lat"])
                    longitude = float(data[0]["lon"])
                    places = search_logdo_map_api(keyword, (latitude, longitude), radius)

                    # Add the place of interest to the list
                    places_of_interest.append({
                        "place_name": name,
                        "latitude": latitude,
                        "longitude": longitude,
                        "places": places or []
                    })
                else:
                    print(f"No geocoding results for {address}.")
            except requests.exceptions.RequestException as e:
                print(f"Geocoding error for {name}: {e}")
        else:
            print("No name found in route data.")

    return places_of_interest


In [5]:
def get_places_from_route(flon, flat, tlon, tlat, keyword, radius):
    """
    Get the route and places of interest along the route.

    Args:
        flon (float): Longitude of the start point.
        flat (float): Latitude of the start point.
        tlon (float): Longitude of the destination point.
        tlat (float): Latitude of the destination point.
        keyword (str): Keyword to search for places of interest.
        radius (int): Radius in kilometers to search for places around each point.

    Returns:
        tuple: A tuple containing route data and places of interest.
    """
    route_data = get_route_data(flon, flat, tlon, tlat)

    if not route_data:
        print("Failed to fetch route data.")
        return None, []

    # Fetch places of interest along the route
    places_of_interest = search_places_of_interest(route_data, keyword, radius)

    return route_data, places_of_interest


In [6]:
if __name__ == "__main__":
    user_query = "อยากกินข้าว"
    user_location = (13.7563, 100.5018)  # กรุงเทพฯ
    user_destination = (14.022788, 99.978337)  # กาญจนบุรี
    radius = 5  # รัศมีการค้นหาสถานที่ 5 กิโลเมตร

    keyword = process_user_query(user_query)
    if keyword:
        print(f"Extracted Keyword: {keyword}")

    flon, flat, tlon, tlat = convert_locations(user_location, user_destination)

    print("\n--- Searching places along the route ---")
    route_data, places_of_interest = get_places_from_route(flon, flat, tlon, tlat, keyword, radius)

    print("\nRoute Data:")
    print(route_data)

    print("\nPlaces of Interest along the route:")
    for place in places_of_interest:
        print(place)


Extracted Keyword: ร้านอาหาร

--- Searching places along the route ---
No geocoding results for ทางหลวงชนบทหมายเลข นฐ.1048.
No geocoding results for ทางหลวงชนบทหมายเลข นฐ.1048.
No geocoding results for ทางหลวงชนบทหมายเลข นฐ.3011.

Route Data:
{'meta': {'status': 200, 'from': {'lon': 100.5018, 'lat': 13.7563}, 'to': {'lon': 99.978337, 'lat': 14.022788}, 'config': 'nullnullnullnull'}, 'data': [{'guide': [{'turn': 4, 'name': 'วงเวียนอนุสาวรีย์ประชาธิปไตย', 'distance': 173, 'interval': 64}, {'turn': 2, 'name': 'ถนนราชดำเนินกลาง', 'distance': 452, 'interval': 107}, {'turn': 2, 'name': 'ถนนราชดำเนินนอก', 'distance': 402, 'interval': 63}, {'turn': 0, 'name': 'ถนนจักรพรรดิพงษ์', 'distance': 24, 'interval': 5}, {'turn': 2, 'name': 'ถนนวิสุทธิกษัตริย์', 'distance': 145, 'interval': 25}, {'turn': 5, 'name': 'สะพานพระราม 8', 'distance': 3345, 'interval': 162}, {'turn': 6, 'name': 'ทางคู่ขนานลอยฟ้าบรมราชชนนี', 'distance': 13000, 'interval': 711}, {'turn': 9, 'name': 'ถนนบรมราชชนนี', 'distance': 214

In [10]:
import folium
from geopy.distance import geodesic
from IPython.display import display

def sort_points(points, start):
    """
    Sort the points based on distance from the starting point sequentially.

    Args:
        points (list): List of (lat, lon) tuples representing points.
        start (tuple): Starting point (lat, lon).

    Returns:
        list: List of sorted points starting from the start point.
    """
    sorted_points = [start]
    points = points.copy()
    current_point = start

    while points:
        next_point = min(points, key=lambda p: geodesic(current_point, p).meters)
        sorted_points.append(next_point)
        points.remove(next_point)
        current_point = next_point

    return sorted_points[1:]  # Exclude the starting point from the result


def show_route_with_connections(user_location, user_destination, route_points):
    """
    Display a map with a route connecting points in order and showing starting/ending points.
    
    Args:
        user_location (tuple): Latitude and longitude of the starting point (lat, lon).
        user_destination (tuple): Latitude and longitude of the destination (lat, lon).
        route_points (list): List of (lat, lon) tuples representing the sequential route.
    
    Returns:
        folium.Map: Map object to be displayed.
    """
    # Create a map centered at the midpoint between the start and end points
    midpoint = (
        (user_location[0] + user_destination[0]) / 2,
        (user_location[1] + user_destination[1]) / 2
    )
    m = folium.Map(location=midpoint, zoom_start=12)

    # Add marker for the starting point (blue)
    folium.Marker(
        location=user_location,
        popup="Start Location",
        icon=folium.Icon(color="blue", icon="info-sign"),
    ).add_to(m)

    # Add marker for the destination (red)
    folium.Marker(
        location=user_destination,
        popup="End Location",
        icon=folium.Icon(color="red", icon="info-sign"),
    ).add_to(m)

    # Add markers for intermediate route points and draw connecting lines
    if route_points:
        # Add intermediate points
        for idx, point in enumerate(route_points):
            folium.Marker(
                location=point,
                popup=f"Point {idx + 1}",
                icon=folium.Icon(color="green", icon="info-sign"),
            ).add_to(m)

        # Draw the route as a connected line
        folium.PolyLine([user_location] + route_points + [user_destination], color="blue", weight=3, opacity=0.8).add_to(m)
    return m
    
def create_map(route_data, places_of_interest, start_location, end_location, sorted_route_points):
    """
    Create and display a map with route data and places of interest.

    Args:
        route_data (dict): Route information containing guide points.
        places_of_interest (list): List of places containing additional information.
        start_location (tuple): Starting point (lat, lon).
        end_location (tuple): Ending point (lat, lon).
        sorted_route_points (list): List of points sorted by proximity to each other.
    """
    # ตรวจสอบว่า route_data มีข้อมูลหรือไม่
    if not route_data:
        print("No route data available.")
        return

    # สร้างแผนที่เริ่มต้นโดยใช้จุดศูนย์กลางระหว่าง start และ end
    midpoint = (
        (start_location[0] + end_location[0]) / 2,
        (start_location[1] + end_location[1]) / 2
    )
    m = folium.Map(location=midpoint, zoom_start=12)

    # เพิ่ม Marker สำหรับ start และ end
    folium.Marker(
        location=start_location,
        popup="Start Location",
        icon=folium.Icon(color="blue", icon="info-sign")
    ).add_to(m)

    folium.Marker(
        location=end_location,
        popup="End Location",
        icon=folium.Icon(color="red", icon="info-sign")
    ).add_to(m)

    # 1. ดึงข้อมูลจาก route_data และเพิ่มเส้นทาง
    route_points = sorted_route_points  # Use sorted_route_points here

    if route_points:
        # เพิ่ม Marker สำหรับจุดตามเส้นทาง
        for idx, point in enumerate(route_points):
            folium.Marker(
                location=point,
                popup=f"Point {idx + 1}",
                icon=folium.Icon(color="green", icon="info-sign")
            ).add_to(m)

        # วาดเส้นทางเชื่อมจุดทั้งหมด
        folium.PolyLine([start_location] + route_points + [end_location], color="blue", weight=3, opacity=0.8).add_to(m)

    # 2. ดึงข้อมูลจาก `places_of_interest` และแสดงสถานที่ที่น่าสนใจ
    places_data = extract_and_return_data_from_places(places_of_interest)

    for data in places_data:
        place_name = data.get('place_name', 'Unknown')
        place_lat = data.get('place_lat', 'Unknown')
        place_lon = data.get('place_lon', 'Unknown')
        data_name = data.get('name', 'Unknown')
        data_lat = data.get('lat', 'Unknown')
        data_lon = data.get('lon', 'Unknown')

        # เครื่องหมายสำหรับ place_name (สีเขียว)
        if place_lat != 'Unknown' and place_lon != 'Unknown':
            folium.Marker(
                location=[place_lat, place_lon],
                popup=f"Place: {place_name}, Lat: {place_lat}, Lon: {place_lon}",
                icon=folium.Icon(color='green', icon='info-sign')
            ).add_to(m)

        # เครื่องหมายสำหรับ data_name (สีส้ม)
        if data_lat != 'Unknown' and data_lon != 'Unknown':
            folium.Marker(
                location=[data_lat, data_lon],
                popup=f"Data: {data_name}, Lat: {data_lat}, Lon: {data_lon}",
                icon=folium.Icon(color='orange', icon='info-sign')
            ).add_to(m)

    # แสดงแผนที่
    display(m)

def extract_and_return_data_from_places(places_of_interest):
    """
    Extract and return data from places_of_interest.

    Args:
        places_of_interest (list): List of places containing place_name and data.

    Returns:
        list: A list of dictionaries containing the name, latitude, and longitude of places.
    """
    extracted_data = []

    for place in places_of_interest:
        place_name = place.get('place_name', 'Unknown')
        place_lat = place.get('latitude', 'Unknown')
        place_lon = place.get('longitude', 'Unknown')

        # ตรวจสอบว่ามีข้อมูลใน 'places' หรือไม่
        places_data = place.get('places', {}).get('data', [])
        if places_data:
            for data in places_data:
                # ดึงข้อมูลที่ต้องการ
                data_name = data.get('name', 'Unknown')
                data_lat = data.get('lat', 'Unknown')
                data_lon = data.get('lon', 'Unknown')

                # เก็บข้อมูลในลิสต์
                extracted_data.append({
                    'place_name': place_name,
                    'place_lat': place_lat,
                    'place_lon': place_lon,
                    'name': data_name,
                    'lat': data_lat,
                    'lon': data_lon
                })
        else:
            # เพิ่มสถานที่หลักหากไม่มีข้อมูลย่อย
            extracted_data.append({
                'place_name': place_name,
                'place_lat': place_lat,
                'place_lon': place_lon
            })

    return extracted_data


# ตัวอย่างข้อมูลจากการเรียกใช้ API
user_query = "อยากกินข้าว"
user_location = (13.7563, 100.5018)  # จุดเริ่มต้น กรุงเทพฯ
user_destination = (14.022788, 99.978337)  # จุดสิ้นสุด กาญจนบุรี
radius = 5  # รัศมีการค้นหาสถานที่ 5 กิโลเมตร

keyword = process_user_query(user_query)  # ฟังก์ชันที่ใช้ในการแยก keyword จาก query
if keyword:
    print(f"Extracted Keyword: {keyword}")

    route_points = [
            (float(place['latitude']), float(place['longitude']))
            for place in places_of_interest
        ]

        # Sort the points based on their proximity to each other starting from the user's location
    sorted_route_points = sort_points(route_points, user_location)


# เรียกใช้ฟังก์ชันเพื่อดึงข้อมูลเส้นทางและสถานที่น่าสนใจ
flon, flat, tlon, tlat = convert_locations(user_location, user_destination)  # ฟังก์ชันที่ใช้แปลงค่าพิกัด

print("\n--- Searching places along the route ---")
route_data, places_of_interest = get_places_from_route(flon, flat, tlon, tlat, keyword, radius)

# สร้างแผนที่
create_map(route_data, places_of_interest, user_location, user_destination, sorted_route_points)


Extracted Keyword: ร้านอาหาร

--- Searching places along the route ---
No geocoding results for ทางหลวงชนบทหมายเลข นฐ.1048.
No geocoding results for ทางหลวงชนบทหมายเลข นฐ.1048.
No geocoding results for ทางหลวงชนบทหมายเลข นฐ.3011.


In [11]:
def extract_and_analyze_data(places_of_interest):
    """
    Extract and analyze data from places_of_interest, considering factors like convenience, opening hours, price, and reviews.

    Args:
        places_of_interest (list): List of places containing place_name and data.

    Returns:
        list: A list of dictionaries containing place analysis including convenience, price, and reviews.
    """
    analyzed_data = []

    for place in places_of_interest:
        place_name = place.get('place_name', 'Unknown')
        place_lat = place.get('latitude', 'Unknown')
        place_lon = place.get('longitude', 'Unknown')

        # ตรวจสอบว่ามีข้อมูลใน 'places' หรือไม่
        places_data = place.get('places', {}).get('data', [])
        if places_data:
            for data in places_data:
                # ดึงข้อมูลที่ต้องการ
                data_name = data.get('name', 'Unknown')
                data_lat = data.get('lat', 'Unknown')
                data_lon = data.get('lon', 'Unknown')
                opening_hours = data.get('opening_hours', 'Unknown')  # เวลาเปิดร้าน
                price_range = data.get('price_range', 'Unknown')  # ราคา
                reviews = data.get('reviews', [])  # รายการรีวิว

                # คำนวณคะแนนจากรีวิว
                average_review_score = 0
                if reviews:
                    total_score = sum(review.get('score', 0) for review in reviews)
                    average_review_score = total_score / len(reviews)

                # คำนวณความสะดวกจากระยะทาง (การให้คะแนนตามระยะทาง)
                distance = calculate_distance(place_lat, place_lon, data_lat, data_lon)  # ฟังก์ชันคำนวณระยะทาง
                convenience_score = max(0, 10 - distance)  # คะแนนความสะดวก (ระยะทาง)

                # เก็บข้อมูลที่ได้
                analyzed_data.append({
                    'place_name': place_name,
                    'place_lat': place_lat,
                    'place_lon': place_lon,
                    'name': data_name,
                    'lat': data_lat,
                    'lon': data_lon,
                    'opening_hours': opening_hours,
                    'price_range': price_range,
                    'average_review_score': average_review_score,
                    'convenience_score': convenience_score
                })
        else:
            # เพิ่มสถานที่หลักหากไม่มีข้อมูลย่อย
            analyzed_data.append({
                'place_name': place_name,
                'place_lat': place_lat,
                'place_lon': place_lon
            })

    return analyzed_data


def calculate_distance(lat1, lon1, lat2, lon2):
    """
    Calculate the distance between two coordinates (in kilometers).

    Args:
        lat1, lon1 (float): Coordinates of the starting point.
        lat2, lon2 (float): Coordinates of the destination point.

    Returns:
        float: The distance between the two points in kilometers.
    """
    from geopy.distance import geodesic
    return geodesic((lat1, lon1), (lat2, lon2)).kilometers

analyzed_places = extract_and_analyze_data(places_of_interest)
for place in analyzed_places:
    print(place)


{'place_name': 'วงเวียนอนุสาวรีย์ประชาธิปไตย', 'place_lat': 13.756689399999999, 'place_lon': 100.50187141917809, 'name': 'สระน้ำร้านอาหาร', 'lat': 15.516138172362986, 'lon': 102.69149463850029, 'opening_hours': 'Unknown', 'price_range': 'Unknown', 'average_review_score': 0, 'convenience_score': 0}
{'place_name': 'วงเวียนอนุสาวรีย์ประชาธิปไตย', 'place_lat': 13.756689399999999, 'place_lon': 100.50187141917809, 'name': 'สระน้ำร้านอาหาร', 'lat': 15.516061279849316, 'lon': 102.69178366997141, 'opening_hours': 'Unknown', 'price_range': 'Unknown', 'average_review_score': 0, 'convenience_score': 0}
{'place_name': 'วงเวียนอนุสาวรีย์ประชาธิปไตย', 'place_lat': 13.756689399999999, 'place_lon': 100.50187141917809, 'name': 'ร้านอาหารเมธาวลัย ศรแดง', 'lat': 13.7560611011446, 'lon': 100.502113871546, 'opening_hours': 'Unknown', 'price_range': 'Unknown', 'average_review_score': 0, 'convenience_score': 9.925706047790106}
{'place_name': 'วงเวียนอนุสาวรีย์ประชาธิปไตย', 'place_lat': 13.756689399999999, 'pl

In [12]:
def recommend_places(places_of_interest, keyword, top_n=10):
    """
    Use LLM to recommend top N places from the search results based on analyzed data.

    Args:
        places_of_interest (list): List of places containing place_name and analyzed data.
        keyword (str): The keyword used for searching.
        top_n (int): Number of top places to recommend.

    Returns:
        str: LLM response with recommendations.
    """
    if not places_of_interest:
        return "ไม่มีสถานที่ที่พบตามคำค้นหา."

    # Prepare the data to be presented to LLM, sorted by the analyzed score
    sorted_places = sorted(places_of_interest, key=lambda x: (
        x.get('average_review_score', 0) * 0.4 + 
        x.get('convenience_score', 0) * 0.3 + 
        (10 - len(x.get('price_range', 'Unknown'))) * 0.2), reverse=True)
    
    # Limit to top N places
    top_places = sorted_places[:top_n]

    places_info = "\n".join([f"{index+1}. {place['name']} (คะแนนรีวิว: {place['average_review_score']}, ความสะดวก: {place['convenience_score']}, ราคา: {place['price_range']})" for index, place in enumerate(top_places)])

    prompt = f"""
    คำค้นหาของผู้ใช้: "{keyword}"
    ต่อไปนี้คือสถานที่ที่พบจากคำค้นหา:
    {places_info}

    กรุณาแนะนำ {top_n} สถานที่ที่ดีที่สุดจากรายการนี้ โดยพิจารณาจากความน่าสนใจ, ความสะดวกในการเข้าถึง, และประเภทของสถานที่, ลักษณะการเดินทาง มีที่จอดรถหรือป่าว, รสชาติอาหารอร่อยหรือป่าว, เหมาะกับเด็กหรือผู้สูงอายุมั้ย.
    โปรดให้ข้อมูลเพิ่มเติมว่าแนะนำจากการวิเคราะห์ใด.
    """

    try:
        # Send the prompt to LLM for recommendation
        response = llm.invoke(prompt)
        return response.content.strip()
    except Exception as e:
        print(f"Error generating recommendation: {e}")
        return "เกิดข้อผิดพลาดในการให้คำแนะนำ."
    

# ตัวอย่างการใช้งาน
if __name__ == "__main__":
    places_of_interest = extract_and_analyze_data(places_of_interest)
    
    print("\n--- LLM Recommendations ---")
    recommendations = recommend_places(places_of_interest, keyword, top_n=10)
    print(recommendations)



--- LLM Recommendations ---
จากข้อมูลที่ให้มาเกี่ยวกับร้านอาหารที่ค้นพบ ฉันจะพิจารณาแค่ความสะดวกในการเข้าถึงเป็นหลัก เนื่องจากคะแนนรีวิวทั้งหมดเป็น 0 และราคายังไม่ทราบ จึงไม่สามารถประเมินรสชาติอาหารหรือความเหมาะสมสำหรับเด็กและผู้สูงอายุได้ ดังนี้คือการจัดอันดับร้านอาหารตามความสะดวกในการเข้าถึง:

1. **ร้านอาหารครัวโสฬส**
   - ความสะดวก: 9.9837
   - ประเภทสถานที่: ไม่ระบุ
   - ข้อมูลเพิ่มเติม: มีความสะดวกสูงที่สุด ควรพิจารณาเป็นตัวเลือกแรก

2. **ร้านอาหารครัวโสฬส**
   - ความสะดวก: 9.9837
   - ประเภทสถานที่: ไม่ระบุ
   - ข้อมูลเพิ่มเติม: ร้านเดียวกันกับข้อแรก

3. **ร้านอาหารศิริโภชนา**
   - ความสะดวก: 9.9764
   - ประเภทสถานที่: ไม่ระบุ
   - ข้อมูลเพิ่มเติม: ความสะดวกสูง ควรพิจารณาเป็นตัวเลือกถัดไป

4. **ร้านอาหารเพ็ญโภชนาข้าวแกง**
   - ความสะดวก: 9.9684
   - ประเภทสถานที่: ไม่ระบุ
   - ข้อมูลเพิ่มเติม: ความสะดวกสูง เหมาะสำหรับผู้ที่ต้องการอาหารง่ายๆ

5. **ร้านอาหารโง่ยลิ้มโภชนา**
   - ความสะดวก: 9.9596
   - ประเภทสถานที่: ไม่ระบุ
   - ข้อมูลเพิ่มเติม: ความสะดวกดี ควรพิจารณา

6. **ร้านอาห

In [13]:
def explain_route_with_llm(route_data):
    route_steps = []
    for instruction in route_data['data'][0]['guide']:
        turn = instruction['turn']
        name = instruction['name']
        distance = instruction['distance']
        interval = instruction['interval']
        
        step = f"เลี้ยวที่ {turn} ไปที่ {name}, ระยะทาง {distance} เมตร, ระยะห่างจากจุดที่แล้ว {interval} เมตร"
        route_steps.append(step)

    route_description = "\n".join(route_steps)

    prompt = f"""
    นี่คือคำแนะนำการเดินทางที่ได้จาก API:
    {route_description}

    กรุณาอธิบายเส้นทางการเดินทางนี้ในรูปแบบภาษาคนที่เข้าใจง่าย
    คำอธิบายควรจะเป็นแบบการแนะนำที่เข้าใจง่ายสำหรับผู้ใช้ที่ไม่เคยเดินทางมาก่อน
    """

    try:
        response = llm.invoke(prompt)
        return response.content.strip()
    except Exception as e:
        print(f"Error generating description: {e}")
        return "เกิดข้อผิดพลาดในการอธิบายการเดินทาง"
    
# ฟังก์ชันการแสดงคำอธิบายการเดินทาง
def display_route_explanation(explanation):
    print("คำอธิบายการเดินทาง:")
    print(explanation)
    
if __name__ == "__main__":
    # เรียกใช้ฟังก์ชันเพื่ออธิบายเส้นทาง
    explanation = explain_route_with_llm(route_data)
    
    # แสดงคำอธิบายการเดินทาง
    display_route_explanation(explanation)


คำอธิบายการเดินทาง:
นี่คือคำแนะนำการเดินทางที่คุณสามารถทำตามได้ง่าย ๆ:

1. เริ่มต้นจากจุดที่คุณอยู่ ให้เลี้ยวที่วงเวียนอนุสาวรีย์ประชาธิปไตย (ระยะทาง 39 เมตร) และเดินไปอีก 11 เมตร
2. จากนั้นเลี้ยวไปที่ถนนราชดำเนินกลาง (ระยะทาง 135 เมตร) โดยให้เดินต่อไปอีก 34 เมตร
3. เลี้ยวไปที่ถนนเชื่อมต่อ (ระยะทาง 55 เมตร) และเดินต่อไป 10 เมตร
4. เลี้ยวไปที่ซอยดำเนินกลางใต้ (ระยะทาง 129 เมตร) และเดินต่อไป 23 เมตร
5. เลี้ยวอีกครั้งไปที่ถนนเชื่อมต่อ (ระยะทาง 104 เมตร) โดยเดินต่อไปอีก 19 เมตร
6. เลี้ยวไปที่ถนนตะนาว (ระยะทาง 48 เมตร) และเดินต่อไป 12 เมตร
7. เลี้ยวไปที่ตรอกเสถียร (ระยะทาง 178 เมตร) และเดินต่อไป 32 เมตร
8. เลี้ยวไปที่ถนนบูรณศาสตร์ (ระยะทาง 143 เมตร) และเดินต่อไป 16 เมตร
9. เลี้ยวไปที่ถนนบุญศิริ (ระยะทาง 190 เมตร) และเดินต่อไป 46 เมตร
10. เลี้ยวไปที่ถนนราชินี (ระยะทาง 356 เมตร) และเดินต่อไป 44 เมตร
11. เลี้ยวไปที่ถนนสมเด็จพระปิ่นเกล้า (ระยะทาง 84 เมตร) และเดินต่อไป 21 เมตร
12. เลี้ยวไปที่สะพานสมเด็จพระปิ่นเกล้า (ระยะทาง 898 เมตร) และเดินต่อไป 90 เมตร
13. เลี้ยวไปที่ถนนสมเด็จพระปิ่นเกล้า (ระย

In [ ]:
import requests
import urllib.parse
from langchain.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import JsonOutputParser
from pydantic import BaseModel
import folium
from geopy.distance import geodesic
from IPython.display import display
from dotenv import load_dotenv
import os

# Define a model for keyword extraction
class SearchKeyword(BaseModel):
    keyword: str
    
load_dotenv()
# Initialize LLM with centralized server configuration
llm = ChatOpenAI(
    model='gpt-4o-mini',
    base_url= os.getenv("url"),  
    api_key= os.getenv("api_key"),  
    max_tokens=1000  
)

In [15]:
def clean_keyword(keyword: str):
    return keyword.strip().lower()

def process_places_of_interest_routes(places_interest):
    try:
        parser = JsonOutputParser(pydantic_object=SearchKeyword)
        format_instructions = """
        คุณต้องกรองคำสำคัญจากคำขอของผู้ใช้.
        คำขอของผู้ใช้คือ: {places_of_interest}
        คำสำคัญที่คุณต้องการหาคือสิ่งที่เกี่ยวข้องกับการค้นหาหรือการกระทำที่ผู้ใช้ต้องการ เช่น:
        - ต้องการทานข้าวหรือแวะพักทานอาหาร: "ร้านอาหาร"
        - ต้องการทานกาแฟ: "ร้านกาแฟ"
        - หากผู้ใช้ต้องการหาห้องน้ำ: "ห้องน้ำ"
        - หากผู้ใช้ต้องการเติมน้ำมัน: "ปั๊มน้ำมัน"
        - หากผู้ใช้ต้องการซื้อของฝาก: "ร้านขายของฝาก"
        - หากผู้ใช้ต้องการซื้อของ: "ร้านสะดวกซื้อ"
        - หากผู้ใช้ต้องการสถานที่แวะพักระหว่างการเดินทาง: "สถานที่พัก"
        กรุณาตอบคำสำคัญที่พบในคำขอนี้ในรูปแบบ JSON ตามตัวอย่างนี้:
        {
            "keyword": "<extracted_keyword>"
        }
        """
        
        prompt = PromptTemplate(
            template="""\
            ## คุณมีหน้าที่กรองคำสำคัญจากคำขอผู้ใช้.

            # คำขอผู้ใช้: {places_of_interest}

            # Your response should be structured as follows:
            {format_instructions}
            """,
            input_variables=["places_of_interest"],
            partial_variables={"format_instructions": format_instructions},
        )

        chain = prompt | llm | parser
        event = chain.invoke({"places_of_interest": places_interest})

        if event:
            keyword = event.get('keyword', '')
            cleaned_keyword = clean_keyword(keyword)
            if cleaned_keyword:
                return cleaned_keyword
    except Exception as e:
        print(f"Error processing user query: {e}")
        return None
    
def convert_locations(user_location, user_destination):
    flat, flon = user_location
    tlat, tlon = user_destination
    return flon, flat, tlon, tlat

In [ ]:
def get_route_data(flon, flat, tlon, tlat):
    """
    Get the route data from the Longdo API based on the given start and end points.

    Args:
        flon (float): Longitude of the start point.
        flat (float): Latitude of the start point.
        tlon (float): Longitude of the destination point.
        tlat (float): Latitude of the destination point.

    Returns:
        dict: Route data returned by the Longdo API.
    """
    try:
        base_url = "https://api.longdo.com/RouteService/json/route/guide?"
        params = {
            'key': os.getenv("key_longdo"),
            'flon': flon,
            'flat': flat,
            'tlon': tlon,
            'tlat': tlat
        }
        full_url = base_url + urllib.parse.urlencode(params)

        response = requests.get(full_url)
        response.raise_for_status()
        route_data = response.json()
    
        return route_data
    except requests.exceptions.RequestException as e:
        print(f"Error fetching route data: {e}")
    return None

def get_lat_lon_from_osm(searched_location):
    """
    Get latitude and longitude from OpenStreetMap Nominatim API.

    Args:
        searched_location (str): The name of the place to search.

    Returns:
        tuple: Latitude and longitude of the searched place, or None if not found.
    """
    url = f'https://nominatim.openstreetmap.org/search?q={urllib.parse.quote(searched_location)}&format=json'
    headers = {'User-Agent': 'MyGeocodingApp/1.0'}
    
    try:
        response = requests.get(url, headers=headers)
        response.raise_for_status()  # จะตรวจสอบว่า HTTP request สำเร็จหรือไม่
        data = response.json()
        
        if data:
            lat = data[0].get('lat')
            lon = data[0].get('lon')
            return lat, lon
    except requests.exceptions.RequestException as e:
        print(f"Error during geocoding: {e}")
    
    return None, None

In [ ]:
def get_places_from_route(route_data):
    """
    Extract place name, latitude, and longitude from route data using OpenStreetMap Nominatim API.
    
    Args:
        route_data (dict): Route data containing the list of guide points.

    Returns:
        list: A list of dictionaries containing place_name, latitude, and longitude.
    """
    places_with_coordinates = []

    # ตรวจสอบว่า route_data มีข้อมูลและ 'data' มีค่าว่างหรือไม่
    if 'data' in route_data and isinstance(route_data['data'], list) and len(route_data['data']) > 0:
        # ใช้ .get() เพื่อดึงข้อมูล guide จาก data[0] โดยไม่ให้เกิดข้อผิดพลาด
        guide_points = route_data['data'][0].get('guide', [])
        
        for point in guide_points:
            name = point.get('name')
            
            if name:
                latitude, longitude = get_lat_lon_from_osm(name)
                
                if latitude and longitude:
                    places_with_coordinates.append({
                        "place_name": name,
                        "latitude": latitude,
                        "longitude": longitude
                    })
    else:
        print("Route data is empty or has an invalid structure.")
    
    return places_with_coordinates


def search_interest_logdo_map_api(keyword, location, radius):
    try:
        base_url = "https://api.longdo.com/mapsearch/json/search?"
        params = {
            'key': os.getenv("key_longdo"),
            'lon': location[1],
            'lat': location[0],
            'radius': radius * 1000,
            'keyword': keyword
        }
        full_url = base_url + urllib.parse.urlencode(params)

        response = requests.get(full_url)
        response.raise_for_status()
        return response.json()
    except requests.exceptions.RequestException as e:
        print(f"Error fetching data from Longdo API: {e}")
    return None

def search_places_of_interest(flon, flat, tlon, tlat, keyword, radius):
    """
    Get route data and search for places of interest along the route.

    Args:
        flon (float): Longitude of the start point.
        flat (float): Latitude of the start point.
        tlon (float): Longitude of the destination point.
        tlat (float): Latitude of the destination point.
        keyword (str): Keyword to search for places of interest.
        radius (int): Radius in kilometers to search for places around each point.

    Returns:
        list: A list containing places of interest along the route.
    """
    route_data = get_route_data(flon, flat, tlon, tlat)
    
    if not route_data:
        print("Failed to fetch route data.")
        return []
    
    # Extract coordinates from route_data
    places_with_coordinates = get_places_from_route(route_data)
    
    # Search for places of interest using the extracted coordinates
    places_of_interest = []
    for place in places_with_coordinates:
        latitude = place["latitude"]
        longitude = place["longitude"]
        found_places = search_interest_logdo_map_api(keyword, (latitude, longitude), radius)
        
        places_of_interest.extend(found_places or [])
    
    return places_of_interest

In [18]:
def sort_points(points, start):
    """
    Sort the points based on distance from the starting point sequentially.

    Args:
        points (list): List of (lat, lon) tuples representing points.
        start (tuple): Starting point (lat, lon).

    Returns:
        list: List of sorted points starting from the start point.
    """
    sorted_points = [start]
    points = points.copy()
    current_point = start

    while points:
        next_point = min(points, key=lambda p: geodesic(current_point, p).meters)
        sorted_points.append(next_point)
        points.remove(next_point)
        current_point = next_point

    return sorted_points[1:]  # Exclude the starting point from the result

# เรื่องทำการแสดงแผนที่ แบ่งเป็นการแสดงเส้นทาง และ การแสดงสถานที่ที่น่าสนใจ
def create_base_map(start_location, end_location):
    """
    Create a basic map centered between start and end locations with markers for both points.
    
    Args:
        start_location (tuple): Starting point (lat, lon).
        end_location (tuple): Ending point (lat, lon).
    
    Returns:
        folium.Map: A map object with markers for start and end locations.
    """
    midpoint = (
        (start_location[0] + end_location[0]) / 2,
        (start_location[1] + end_location[1]) / 2
    )
    m = folium.Map(location=midpoint, zoom_start=12)

    folium.Marker(
        location=start_location,
        popup="Start Location",
        icon=folium.Icon(color="blue", icon="info-sign")
    ).add_to(m)

    folium.Marker(
        location=end_location,
        popup="End Location",
        icon=folium.Icon(color="red", icon="info-sign")
    ).add_to(m)

    return m

def create_map(route_data, places_of_interest, start_location, end_location, sorted_route_points):
    """
    Create and display a map with route data and places of interest.

    Args:
        route_data (dict): Route information containing guide points.
        places_of_interest (list): List of places containing additional information.
        start_location (tuple): Starting point (lat, lon).
        end_location (tuple): Ending point (lat, lon).
        sorted_route_points (list): List of points sorted by proximity to each other.
    """
    if not route_data:
        print("No route data available.")
        return

    m = create_base_map(start_location, end_location)

    route_points = sorted_route_points  # Use sorted_route_points here

    if route_points:
        for idx, point in enumerate(route_points):
            folium.Marker(
                location=point,
                popup=f"Point {idx + 1}",
                icon=folium.Icon(color="green", icon="info-sign")
            ).add_to(m)

        folium.PolyLine([start_location] + route_points + [end_location], color="blue", weight=3, opacity=0.8).add_to(m)

    places_data = extract_and_return_data_from_places(places_of_interest)

    for data in places_data:
        place_name = data.get('place_name', 'Unknown')
        place_lat = data.get('place_lat', 'Unknown')
        place_lon = data.get('place_lon', 'Unknown')
        
        if place_lat != 'Unknown' and place_lon != 'Unknown':
            folium.Marker(
                location=[place_lat, place_lon],
                popup=f"Place: {place_name}, Lat: {place_lat}, Lon: {place_lon}",
                icon=folium.Icon(color='green', icon='info-sign')
            ).add_to(m)

    display(m)

In [19]:
def extract_and_return_data_from_places(places_of_interest):
    """
    Extract and return data from places_of_interest.

    Args:
        places_of_interest (list): List of places containing place_name and data.

    Returns:
        list: A list of dictionaries containing the name, latitude, and longitude of places.
    """
    extracted_data = []

    for place in places_of_interest:
        place_name = place.get('place_name', 'Unknown')
        place_lat = place.get('latitude', 'Unknown')
        place_lon = place.get('longitude', 'Unknown')

        # ตรวจสอบว่ามีข้อมูลใน 'places' หรือไม่
        places_data = place.get('places', {}).get('data', [])
        if places_data:
            for data in places_data:
                # ดึงข้อมูลที่ต้องการ
                data_name = data.get('name', 'Unknown')
                data_lat = data.get('lat', 'Unknown')
                data_lon = data.get('lon', 'Unknown')

                # เก็บข้อมูลในลิสต์
                extracted_data.append({
                    'place_name': place_name,
                    'place_lat': place_lat,
                    'place_lon': place_lon,
                    'name': data_name,
                    'lat': data_lat,
                    'lon': data_lon
                })
        else:
            # เพิ่มสถานที่หลักหากไม่มีข้อมูลย่อย
            extracted_data.append({
                'place_name': place_name,
                'place_lat': place_lat,
                'place_lon': place_lon
            })

    return extracted_data

# ทำการวิเคราะห์สถานที่
def extract_and_analyze_data(places_of_interest):
    """
    Extract and analyze data from places_of_interest, considering factors like convenience, opening hours, price, and reviews.

    Args:
        places_of_interest (list): List of places containing place_name and data.

    Returns:
        list: A list of dictionaries containing place analysis including convenience, price, and reviews.
    """
    analyzed_data = []

    if not places_of_interest:
        print("No places of interest to analyze.")
        return analyzed_data

    for place in places_of_interest:
        place_name = place.get('place_name', 'Unknown')
        place_lat = place.get('latitude', 'Unknown')
        place_lon = place.get('longitude', 'Unknown')

        # ตรวจสอบว่ามีข้อมูลใน 'places' หรือไม่
        places_data = place.get('places', {}).get('data', [])
        if places_data:
            for data in places_data:
                # ดึงข้อมูลที่ต้องการ
                data_name = data.get('name', 'Unknown')
                data_lat = data.get('lat', 'Unknown')
                data_lon = data.get('lon', 'Unknown')
                opening_hours = data.get('opening_hours', 'Unknown')  # เวลาเปิดร้าน
                price_range = data.get('price_range', 'Unknown')  # ราคา
                reviews = data.get('reviews', [])  # รายการรีวิว

                # คำนวณคะแนนจากรีวิว
                average_review_score = 0
                if reviews:
                    total_score = sum(review.get('score', 0) for review in reviews)
                    average_review_score = total_score / len(reviews)

                # คำนวณความสะดวกจากระยะทาง (การให้คะแนนตามระยะทาง)
                distance = calculate_distance(place_lat, place_lon, data_lat, data_lon)  # ฟังก์ชันคำนวณระยะทาง
                convenience_score = max(0, 10 - distance)  # คะแนนความสะดวก (ระยะทาง)

                # เก็บข้อมูลที่ได้
                analyzed_data.append({
                    'place_name': place_name,
                    'place_lat': place_lat,
                    'place_lon': place_lon,
                    'name': data_name,
                    'lat': data_lat,
                    'lon': data_lon,
                    'opening_hours': opening_hours,
                    'price_range': price_range,
                    'average_review_score': average_review_score,
                    'convenience_score': convenience_score
                })
        else:
            # เพิ่มสถานที่หลักหากไม่มีข้อมูลย่อย
            analyzed_data.append({
                'place_name': place_name,
                'place_lat': place_lat,
                'place_lon': place_lon
            })

    return analyzed_data


In [22]:
def calculate_distance(lat1, lon1, lat2, lon2):
    """
    Calculate the distance between two coordinates (in kilometers).

    Args:
        lat1, lon1 (float): Coordinates of the starting point.
        lat2, lon2 (float): Coordinates of the destination point.

    Returns:
        float: The distance between the two points in kilometers.
    """
    from geopy.distance import geodesic
    return geodesic((lat1, lon1), (lat2, lon2)).kilometers

def recommend_places(places_of_interest, keyword, top_n=10):
    """
    Use LLM to recommend top N places from the search results based on analyzed data.

    Args:
        places_of_interest (list): List of places containing place_name and analyzed data.
        keyword (str): The keyword used for searching.
        top_n (int): Number of top places to recommend.

    Returns:
        str: LLM response with recommendations.
    """
    if not places_of_interest:
        return "ไม่มีสถานที่ที่พบตามคำค้นหา."

    # Prepare the data to be presented to LLM, sorted by the analyzed score
    sorted_places = sorted(places_of_interest, key=lambda x: (
        x.get('average_review_score', 0) * 0.4 + 
        x.get('convenience_score', 0) * 0.3 + 
        (10 - len(x.get('price_range', 'Unknown'))) * 0.2), reverse=True)
    
    # Limit to top N places
    top_places = sorted_places[:top_n]

    places_info = "\n".join([f"{index+1}. {place['name']} (คะแนนรีวิว: {place['average_review_score']}, ความสะดวก: {place['convenience_score']}, ราคา: {place['price_range']})" for index, place in enumerate(top_places)])

    prompt = f"""
    คำค้นหาของผู้ใช้: "{keyword}"
    ต่อไปนี้คือสถานที่ที่พบจากคำค้นหา:
    {places_info}

    กรุณาแนะนำ {top_n} สถานที่ที่ดีที่สุดจากรายการนี้ โดยพิจารณาจากความน่าสนใจ, ความสะดวกในการเข้าถึง, และประเภทของสถานที่, ลักษณะการเดินทาง มีที่จอดรถหรือป่าว, รสชาติอาหารอร่อยหรือป่าว, เหมาะกับเด็กหรือผู้สูงอายุมั้ย.
    โปรดให้ข้อมูลเพิ่มเติมว่าแนะนำจากการวิเคราะห์ใด.
    """

    try:
        # Send the prompt to LLM for recommendation
        response = llm.invoke(prompt)
        return response.content.strip()
    except Exception as e:
        print(f"Error generating recommendation: {e}")
        return "เกิดข้อผิดพลาดในการให้คำแนะนำ."
    
def explain_route_with_llm(route_data):
    route_steps = []
    
    # ตรวจสอบว่า 'data' และ 'guide' มีอยู่ใน route_data ก่อนเข้าถึง
    if 'data' in route_data and len(route_data['data']) > 0 and 'guide' in route_data['data'][0]:
        for instruction in route_data['data'][0]['guide']:
            turn = instruction.get('turn', 'ไม่ระบุ')
            name = instruction.get('name', 'ไม่ระบุ')
            distance = instruction.get('distance', 'ไม่ระบุ')
            interval = instruction.get('interval', 'ไม่ระบุ')
            
            step = f"เลี้ยวที่ {turn} ไปที่ {name}, ระยะทาง {distance} เมตร, ระยะห่างจากจุดที่แล้ว {interval} เมตร"
            route_steps.append(step)

        route_description = "\n".join(route_steps)

        prompt = f"""
        นี่คือคำแนะนำการเดินทางที่ได้จาก API:
        {route_description}

        กรุณาอธิบายเส้นทางการเดินทางนี้ในรูปแบบภาษาคนที่เข้าใจง่าย
        คำอธิบายควรจะเป็นแบบการแนะนำที่เข้าใจง่ายสำหรับผู้ใช้ที่ไม่เคยเดินทางมาก่อน
        """

        try:
            # ตรวจสอบว่า LLM พร้อมใช้งาน
            if hasattr(llm, 'invoke'):
                response = llm.invoke(prompt)
                return response.content.strip()
            else:
                print("Error: llm.invoke is not available")
                return "ไม่สามารถเรียกใช้งาน LLM ได้"
        except Exception as e:
            print(f"Error generating description: {e}")
            return "เกิดข้อผิดพลาดในการอธิบายการเดินทาง"
    else:
        return "ข้อมูลการเดินทางไม่สมบูรณ์หรือไม่ถูกต้อง"

# ฟังก์ชันการแสดงคำอธิบายการเดินทาง
def display_route_explanation(route_data):
    explanation = explain_route_with_llm(route_data)
    print("คำอธิบายการเดินทาง:")
    print(explanation)

In [24]:
places_interest = "อยากกินข้าว"
user_location = (13.7563, 100.5018)  # จุดเริ่มต้น กรุงเทพฯ
user_destination = (14.022788, 99.978337)  # จุดสิ้นสุด กาญจนบุรี
radius = 5  # รัศมีการค้นหาสถานที่ 5 กิโลเมตร

# ใช้ฟังก์ชัน process_places_of_interest_routes ในการแยก keyword จาก query
keyword = process_places_of_interest_routes(places_interest)
if keyword:
    print(f"Extracted Keyword: {keyword}")

    # เรียกฟังก์ชัน search_places_of_interest เพื่อค้นหาสถานที่ตาม keyword
    places_of_interest = search_places_of_interest(user_location[1], user_location[0], user_destination[1], user_destination[0], keyword, radius)
    
    if places_of_interest:
        route_points = [
            (float(place['latitude']), float(place['longitude']))  # แก้ไขให้ใช้ค่าจาก places_of_interest
            for place in places_of_interest
            if isinstance(place, dict) and 'latitude' in place and 'longitude' in place
        ]
        
        # Sort the points based on their proximity to each other starting from the user's location
        sorted_route_points = sort_points(route_points, user_location)


        # เรียกใช้ฟังก์ชันเพื่อดึงข้อมูลเส้นทางและสถานที่น่าสนใจ
        flon, flat, tlon, tlat = convert_locations(user_location, user_destination)  # ฟังก์ชันที่ใช้แปลงค่าพิกัด
        route_data = {'flon': flon, 'flat': flat, 'tlon': tlon, 'tlat': tlat, 'keyword': keyword, 'radius': radius}

        print("--- Searching places along the route ---")
        places_of_interest = get_places_from_route(route_data)  # เรียกฟังก์ชันด้วย route_data
        if places_of_interest:
            analyzed_places = extract_and_analyze_data(places_of_interest)
        else:
            print("No places of interest found.")
        
        # สร้างแผนที่
        create_map(route_data, places_of_interest, user_location, user_destination, sorted_route_points)

        print("--- LLM Recommendations ---")
        recommendations = recommend_places(places_of_interest, keyword, top_n=10)
        print(recommendations)

        explanation = explain_route_with_llm(route_data)  
        # แสดงคำอธิบายการเดินทาง
        display_route_explanation(explanation)
    else:
        print("No places found matching the search.")
else:
    print("Error processing the input.")

Extracted Keyword: ร้านอาหาร
--- Searching places along the route ---
Route data is empty or has an invalid structure.
No places of interest found.


--- LLM Recommendations ---
ไม่มีสถานที่ที่พบตามคำค้นหา.
คำอธิบายการเดินทาง:
ข้อมูลการเดินทางไม่สมบูรณ์หรือไม่ถูกต้อง
